In [1]:
import pymupdf4llm
import markdown
from bs4 import BeautifulSoup
md_text_6 = pymupdf4llm.to_markdown("6.pdf")

In [2]:
import pandas as pd
import markdown
from bs4 import BeautifulSoup

def extract_sections_to_dataframe(markdown_text):
    lines = markdown_text.split('\n')
    
    current_l1 = None  # Kapitel
    current_l2 = None  # Kategorie
    current_l3 = None  # Kriterium
    current_paragraph = []
    data_rows = []
    
    for line in lines:
        if line.strip().startswith('#'):
            # Save current section if exists
            if current_l3 and current_paragraph:
                text = '\n'.join(current_paragraph).strip()
                if "**Supplementary Information**" in text:
                    parts = text.split("\n**Supplementary Information**\n")
                    beschreibung = BeautifulSoup(markdown.markdown(parts[0]), "html.parser").get_text()
                    #weitere_inf = BeautifulSoup(markdown.markdown(parts[1]), "html.parser").get_text()
                    try:
                        weitere_inf = BeautifulSoup(markdown.markdown(parts[1]), "html.parser").get_text()
                    except Exception as e:
                        weitere_inf = ""
                        print("failed")
                        print(e)
                    data_rows.append({
                        'Kapitel': current_l1,
                        'Kategorie': current_l2,
                        'Kriterium': current_l3,
                        'Beschreibung': beschreibung,
                        'Weitere informationen': weitere_inf
                    })
                current_paragraph = []
            
            level = len(line) - len(line.lstrip('#'))
            heading_text = line.lstrip('#').strip()
            
            if level == 1:
                current_l1 = heading_text
            elif level == 2:
                current_l2 = heading_text
            elif level == 3:
                current_l3 = heading_text
        
        elif current_l3 and line.strip():
            current_paragraph.append(line.strip())
    
    # Process last section
    if current_l3 and current_paragraph:
        text = '\n'.join(current_paragraph).strip()
        if "**Supplementary Information**" in text:
            parts = text.split("\n**Supplementary Information**\n")
            beschreibung = BeautifulSoup(markdown.markdown(parts[0]), "html.parser").get_text()
            try:
                weitere_inf = BeautifulSoup(markdown.markdown(parts[1]), "html.parser").get_text()
            except Exception as e:
                weitere_inf = ""
                print("failed")
                print(e)
            data_rows.append({
                'Kapitel': current_l1,
                'Kategorie': current_l2,
                'Kriterium': current_l3,
                'Beschreibung': beschreibung,
                'Weitere informationen': weitere_inf
            })
    
    return pd.DataFrame(data_rows)


# Create DataFrame
df6 = extract_sections_to_dataframe(md_text_6)

failed
list index out of range
failed
list index out of range
failed
list index out of range


In [3]:
md_text_5 = pymupdf4llm.to_markdown("5.pdf")

In [4]:
def extract_sections_to_dataframe(markdown_text):
    lines = markdown_text.split('\n')
    
    current_l1 = None  # Kapitel
    current_l2 = None  # Kriterium
    current_paragraph = []
    data_rows = []
    
    def process_paragraph():
        if current_l2 and current_paragraph:
            text = '\n'.join(current_paragraph).strip()
            beschreibung = BeautifulSoup(markdown.markdown(text), "html.parser").get_text()
            
            data_rows.append({
                'Kapitel': current_l1,
                'Kriterium': current_l2,
                'Beschreibung': beschreibung
            })
    
    for line in lines:
        if line.strip().startswith('#'):
            # Process current paragraph before moving to new section
            process_paragraph()
            current_paragraph = []
            
            level = len(line) - len(line.lstrip('#'))
            heading_text = line.lstrip('#').strip()
            
            if level == 1:
                current_l1 = heading_text
            elif level == 2:
                current_l2 = heading_text
        
        elif current_l2 and line.strip():
            current_paragraph.append(line.strip())
    
    # Process the final paragraph
    process_paragraph()
    
    return pd.DataFrame(data_rows)

# Create DataFrame
df5 = extract_sections_to_dataframe(md_text_5)

# Save to Excel
#df.to_excel('/Users/marcrodig/Development/Side projects/output.xlsx', index=False)

In [5]:
with pd.ExcelWriter('/Users/marcrodig/Development/Side projects/aic4.xlsx', engine='openpyxl') as writer:
    df5.to_excel(writer, sheet_name='Kapitel 5 ', index=False)
    df6.to_excel(writer, sheet_name='Kapitel 6', index=False)